## Name: Optimal bounded variable value estimation in sum-product networks
### Date: 29/10/2024
### Status: Works! Interesting that we iterate over the terminals and the constraints with different ways to optimize.
### Idea: 
The task is, given a multi-linear function in the form of a product of terminal nodes and specific bounds on (probability) variables, find the optimal value of the variables that maximize the function.


Spefically, we want $$ \mathop{argmax}_{a,b,c}{F(a,b,c)} = \mathop{argmax}_{a,b,c}{\prod_{i}^{N}T_{i}(a,b,c)}  $$ where $ a \in [l_a, u_a], b \in [l_b, u_b], ... $.
The idea is that we can try to maximize this function by maximizing each terminal separately but as a linear system altogether.

We want to maximize  the system $$ T_1 = 1, T_2 = 1, ..., T_N = 1 $$

Let's take for example the case where $$ F = T_1 * T_2 = (a) * (1-a)*b $$ This would become:  $$ a = 1, (1-a) * b = 1 $$ in which case it is a multi-linear system.

We introduce the variable $ a' = 1-a $ alongside the constraint $a' + a = 1$.

Now we have a set of linear equations and linear constraints.

We use a solver that iteratevly search for the bounded solution of this function by calculating the weighted residuals best fit on this system. We make it so, that the terminal functions are resolved by taking the product of the values, while the introduced constraints by taking the same of the values of the variables. We also, greatly outweight the constraints (1000:1 the constraints:terminal weight ratio), to make sure the solutions are in constraints.


### Results:
It works!
Not sure if it usable anywhere. Could have done the same with gradient descent and constraints for sure.

In [1]:
import sympy
import numpy as np

A, B, C, D, E, F, G, H, I, J, K, L, M, N, O = sympy.symbols('A, B, C, D, E, F, G, H, I, J, K, L, M, N, O')
expression = eval("((A & B & C & D) | (A & B))")
truth_table = list(sympy.logic.boolalg.truth_table(expression, [A, B, C, D])) 
X = np.array([x[0] for x in truth_table])
y = np.array([bool(x[-1]) for x in truth_table]).astype(int)
print(X, y)

[[0 0 0 0]
 [0 0 0 1]
 [0 0 1 0]
 [0 0 1 1]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 1 1 1]] [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]


In [2]:
from sklearn.tree import DecisionTreeClassifier, export_text
dt = DecisionTreeClassifier()
dt.fit(X,y)
dt.score(X,y)

1.0

In [3]:
print(export_text(dt, feature_names=['A', 'B', 'C', 'D']))

|--- A <= 0.50
|   |--- class: 0
|--- A >  0.50
|   |--- B <= 0.50
|   |   |--- class: 0
|   |--- B >  0.50
|   |   |--- class: 1



In [4]:
from scipy.optimize import lsq_linear
import numpy as np
import pandas as pd

# TRIVIAL CASE WITH MONOTONICITY
# correct would be all max

# t1 = (a * b * c * d), t2 =(a * d)
T = np.array([[1,1,1, 1], [1,0,0,1]])
y = np.array([1,1])

lb = [0.3, 0.3, 0.3, 0.3]
ub = [0.4, 0.4, 0.4, 0.4]

res = lsq_linear(T, y, bounds=(np.log(lb), np.log(ub)), lsmr_tol='auto', verbose=0)
np.exp(res.x)


array([0.4, 0.4, 0.4, 0.4])

In [14]:
# NON trivial CASE WITH b and not b
# This would depend on the bounds of a.
# With t1 = a*b and t2 = a*(1-b) we have sum(t1, t2) = ab + a(1-b) = ab + a -ab = a
# So we just need any point with max bound in a

# t1 = a*b, t2 = a*(1-b)
T = np.array([[1,1], [1,-1]])
y = np.array([1,1])
from scipy.optimize import lsq_linear
lb = [0.3, 0.3]
ub = [0.4, 0.4]

bounds = np.vstack((lb, ub)).T

res = lsq_linear(T, y, bounds=(np.log(lb), np.log(ub)), lsmr_tol='auto', verbose=0)
print(f"Best res: {np.exp(res.x)}")


for x1 in bounds[0]:
    for x2 in bounds[1]:
        y = np.exp((T@np.array([x1,x2])).sum())
        print(f"Func({x1} - {x2}) = {y:0f}")

Best res: [0.4 0.4]
Func(0.3 - 0.3) = 1.822119
Func(0.3 - 0.4) = 1.822119
Func(0.4 - 0.3) = 2.225541
Func(0.4 - 0.4) = 2.225541


In [90]:
def evaluate_function(terminals:np.ndarray, values:np.ndarray):
    values_augmented = np.concatenate((values, 1-values))
    total = 0
    for t in terminals:
        indices = np.where(t>0)[0]
        prod = values_augmented[indices].prod()
        total = prod
    return total

def get_optimal_value_from_edge_cases(T, bounds):
    res_all = []
    num_true_vars = len(bounds)
    num_terminals = T.shape[0] - num_true_vars
    print("num_terminals", num_terminals, T[:num_terminals])
    for a in bounds[0]:
        for b in bounds[1]:
            for c in bounds[2]:
                cur_val = np.array([a,b,c])
                res = evaluate_function(T[:num_terminals], cur_val)
                res_all.append(cur_val.tolist() + [res])
                
    import pandas as pd
    res_all = pd.DataFrame(res_all, columns=['a', 'b', 'c', 'val'])
    print(res_all.sort_values('val'))
    return res_all.sort_values('val').iloc[-1]

  


def weighted_func(x, T, focus='function', weight_important=100):
    num_actual_vars = int(x.shape[0]/2)
    num_terminals = T.shape[0] - num_actual_vars
    
    y = np.ones_like(T.shape[0])
    
    y_pred = []
    for t_index, t in enumerate(T):
      indices = np.where(t>0)[0]
      if t_index < num_terminals:
        y_ = x[indices].prod()
      else:
        y_ = x[indices].sum()
      y_pred.append(y_)
    y_pred = np.array(y_pred)

    residuals = y - y_pred
    
    if focus == 'balanced':
      weights = np.ones_like(y)
    elif focus == 'constraints':
      weights = np.array([1 for _ in range(num_terminals)] + [weight_important for _ in range(num_actual_vars)])
    elif focus == 'function':
      weights = np.array([weight_important for _ in range(num_terminals)] + [1 for _ in range(num_actual_vars)])
    else:
      raise NotImplementedError(f'{focus} not understood')
    
  
    weighted_residuals = weights*residuals
    return weighted_residuals
  
  

T = np.array([ 
              [0,0,0,1,0,1], # t1 = (1-a)*(1-c) 
              [1,1,0,0,0,1], # t2 = a*b*(1-c)
              [1,0,0,1,0,0], # c1: a + (1-a) = 1
              [0,1,0,0,1,0], # c2: b + (1-b) = 1
              [0,0,1,0,0,1],# c3: c + (1-c) = 1
            ])

lb = [0.7, 0.2, 0.3]
ub = [0.8, 0.6, 0.4]
bounds = np.vstack((lb, ub)).T

print(get_optimal_value_from_edge_cases(T, bounds))


num_terminals 2 [[0 0 0 1 0 1]
 [1 1 0 0 0 1]]
     a    b    c    val
1  0.7  0.2  0.4  0.084
5  0.8  0.2  0.4  0.096
0  0.7  0.2  0.3  0.098
4  0.8  0.2  0.3  0.112
3  0.7  0.6  0.4  0.252
7  0.8  0.6  0.4  0.288
2  0.7  0.6  0.3  0.294
6  0.8  0.6  0.3  0.336
a      0.800
b      0.600
c      0.300
val    0.336
Name: 6, dtype: float64


In [109]:

def weighted_func(x, T, focus='function', weight_important=100):
    num_actual_vars = int(x.shape[0]/2)
    num_terminals = T.shape[0] - num_actual_vars
    y = np.ones_like(T.shape[0])

    y_pred = []
    for t_index, t in enumerate(T):
      indices = np.where(t>0)[0]
      if t_index < num_terminals:
        y_ = x[indices].prod()
      else:
        y_ = x[indices].sum()
      y_pred.append(y_)
    y_pred = np.array(y_pred)

    residuals = y - y_pred
    
    if focus == 'balanced':
      weights = np.ones_like(y)
    elif focus == 'constraints':
      weights = np.array([1 for _ in range(num_terminals)] + [weight_important for _ in range(num_actual_vars)])
    elif focus == 'function':
      weights = np.array([weight_important for _ in range(num_terminals)] + [1 for _ in range(num_actual_vars)])
    # make sure we focuson constraints
    else:
      raise NotImplementedError(f'{focus} not understood')
    
    
    # make sure we focus on maximizing the target
    
    
    
    weighted_residuals = weights*residuals
    return weighted_residuals
  
  
from scipy.optimize import  least_squares



T = np.array([ 
              [1,0,0,0,0,0], # t1 = a
              [0,1,1,1,0,0], # t2 = (1-a)*b*c
              [1,0,0,1,0,0], # c1: a + (1-a) = 1
              [0,1,0,0,1,0], # c2: b + (1-b) = 1
              [0,0,1,0,0,1],# c3: c + (1-c) = 1
            ])

lb = [0.7, 0.2, 0.3]
ub = [0.8, 0.6, 0.4]

bounds = np.vstack((lb, ub)).T

print(f'Lower bounds: {lb[:3]}')
print(f'Upper bounds: {ub[:3]}')

print('Optimal result:')
print(get_optimal_value_from_edge_cases(T, bounds).tolist())


lb = lb + [1 - ub_ for ub_ in ub]
ub = ub + [1 - lb_ for lb_ in lb[:int(len(lb)/2)]]

x0 = np.array([item for item in lb[:3]] + [1-item for item in lb[:3]])
# print('x0', x0)

res = least_squares(weighted_func, x0=x0, bounds=(lb, ub), verbose=0, kwargs={'T':T, "focus":'constraints',  "weight_important":1000})
print('NE result:')
print(np.round(res.x[:3], decimals=3))



Lower bounds: [0.7, 0.2, 0.3]
Upper bounds: [0.8, 0.6, 0.4]
Optimal result:
num_terminals 2 [[1 0 0 0 0 0]
 [0 1 1 1 0 0]]
     a    b    c    val
4  0.8  0.2  0.3  0.012
5  0.8  0.2  0.4  0.016
0  0.7  0.2  0.3  0.018
1  0.7  0.2  0.4  0.024
6  0.8  0.6  0.3  0.036
7  0.8  0.6  0.4  0.048
2  0.7  0.6  0.3  0.054
3  0.7  0.6  0.4  0.072
[0.7, 0.6, 0.4, 0.07200000000000001]
NE result:
[0.7 0.6 0.4]
